# Assemble the final Dataframe from tarpmir bindingsites

## Read in bindingsite & Ensembl data

In [1]:
%load_ext autoreload
%autoreload 2
from helper_fcts import *
import sqlite3
import os
from pathlib import Path

path_tarp = Path('C:/Users/Lena/Documents/Master big files/TarPmiR_Windows/data/')
path_gdc = Path('C:/Users/Lena/Documents/Master big files/GDC_download_legacy/')
path2 = Path('C:/Users/Lena/Documents/auf mega gesyncte Dokumente/Uni/Master/Semester 5/Masterarbeit/')
fig_path = Path('Dokumente/plots/')

In [4]:
#executed once
all_mapping, ids = read_in_ids2(path_tarp/'all_mapping_ids.fasta') # mapping gene, transcript, exon ids, all
chrom_exon_starts = read_in_fasta2(path_tarp/'all_exon_start.fasta')
chrom_exon_ends = read_in_fasta2(path_tarp/'all_exon_end.fasta')
exon_starts, exon_ends = calc_exon_data(chrom_exon_starts, chrom_exon_ends)#TODO remove this & instead work with exon_info? slower :(

#df_main: bs_id - chr_start - chr_end - miRNA_name - bs_seed - bs_len - tarp_output - transcript_id
#df_gene: strand - chrom - gene_id - gene_name - transcript_id (1-N)

In [76]:
binding_sites = parse_tarp_bs(path_tarp/'new_input/input_miRNA.fasta_exampleA1BG_mRNA.txt.bp')
binding_sites.to_sql('bs_tap', con=conn)

C:\Users\Lena\Miniconda3\envs\thesis\lib\site-packages\pandas\core\generic.py:2653: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [4]:
#old TODO make efficient
#old : 580 split+pandas+sqlite
#new: 90+348 #8000 with , \bs_id integer primary key autoincrement \+ split+sqlite
#parse miRNA and mRNA seed from sequence
df_main = binding_sites.copy()
miRNA_sequences = parse_seq(path_tarp/'new_input'/'input_miRNA.fasta')
df_main['miRNA_seed'] = df_main.apply(lambda row: miRNA_sequences[row['miRNA']], axis=1)
mRNA_sequences = parse_seq(path_tarp/'new_input'/'exampleA1BG_mRNA.txt')
df_main['mRNA_bs_seq'] = df_main.apply(lambda row: mRNA_sequences[row['mRNA']][row['bs_start']:row['bs_end']], axis=1)

In [79]:
#DELETE
conn = sqlite3.connect('db.db')
c = conn.cursor()
c.execute("DROP TABLE main;")
conn.commit()

In [81]:
#execute once to create database
conn = sqlite3.connect('db.db')
c = conn.cursor()
c.execute("CREATE TABLE main ( \
    miRNA varchar(255), \
    mRNA varchar(255), \
    binding_site varchar(255), \
    binding_probability float, \
    energy float, \
    seed int, \
    accessibility float, \
    AU_content float, \
    PhyloP_Stem float, \
    PyloP_Flanking float, \
    m_or_e float, \
    number_of_pairings int, \
    binding_region_length int, \
    longest_consecutive_pairings int, \
    position_of_longest_consecutive_pairings int, \
    pairings_in_3prime_end int, \
    difference_of_pairings_between_seed_and_3prime_end int \
);")
conn.commit()

In [82]:
#run this for every tarpmir bp file
#for filename in os.listdir(tarp_pred):
#input file should be without header
#bs_id is automatically generated primary key
import subprocess
def add_to_sqlite_db(csv_file, seperator, tablename):
    res = subprocess.run(['sqlite3',
                    str(path2/'db.db'),
                    '-cmd',
                    '.mode csv',
                    f'.separator {seperator}',
                    '.import "'+str(csv_file).replace("\\","\\\\")+'" '+tablename],
                    capture_output=True)
    return res
csv_file = path_tarp/'new_input'/'input_miRNA.fasta_exampleA1BG_mRNA.txt.bp'
res = add_to_sqlite_db(csv_file, "\\t",'main')

In [68]:
#new table bs with split binding site start+end rows

qry3 = ''' CREATE TABLE bs as
  SELECT main.bs_id, miRNA, mRNA, start, end FROM
  main INNER JOIN (
    SELECT substr(binding_site, 1, pos - 1) AS start,
           substr(substr(binding_site, pos + 1), 1, pos - 1) AS end,
           bs_id
      FROM (
            SELECT  binding_site, 
                    bs_id, 
                    instr(binding_site || ',', ',') AS pos
            FROM main)) AS sep
    ON main.bs_id = sep.bs_id;
'''
c.execute(qry3)
#c.execute("DROP TABLE main;") #keep original table with mapping to original tarpmir output
conn.commit()
c.fetchall()

[]

In [74]:
#print bs
qry = '''
    select * from main
    '''
c.execute(qry)
conn.commit()
c.fetchall()
pd.read_sql_query(qry,conn)

,miRNA,mRNA,binding_site,binding_probability,energy,seed,accessibility,AU_content,PhyloP_Stem,PyloP_Flanking,m_or_e,number_of_pairings,binding_region_length,longest_consecutive_pairings,position_of_longest_consecutive_pairings,pairings_in_3prime_end,difference_of_pairings_between_seed_and_3prime_end,bs_id
0,miRNA,mRNA,binding_site,binding_probability,energy,seed,accessibility,AU_content,PhyloP_Stem,PyloP_Flanking,m/e,number_of_pairings,binding_region_length,longest_consecutive_pairings,position_of_longest_consecutive_pairings,pairings_in_3prime_end,difference_of_pairings_between_seed_and_3prime...,1
1,hsa-let-7a-5p,ENST00000600966,"396,416",0.692308,-15.4,0,0.00337109,0.368,-0.568203,0.198163,-7.75705,16,20,10,10,6,1,2
2,hsa-let-7a-2-3p,ENST00000600966,"52,75",0.923077,-23.2,1,9.40998e-05,0.426,0.844482,-0.0950648,-5.58516,18,23,7,1,6,1,3
3,hsa-let-7b-5p,ENST00000600966,"775,794",0.692308,-21.3,0,9.22052e-06,0.353,1.25308,0.0152552,-8.61389,16,19,9,1,7,2,4
4,hsa-let-7b-5p,ENST00000600966,"126,143",0.692308,-18.5,0,1.17558e-06,0.294,1.30246,-0.0926582,-9.58239,15,17,9,1,6,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74140,hsa-miR-12128,ENST00000598345,"243,269",0.538462,-21.7,0,0.0158677,0.324,0,0,-11.1417,18,26,5,21,5,1,74141
74141,hsa-miR-12128,ENST00000598345,"193,213",0.692308,-20.3,0,0.000161199,0.294,0,0,-17.0305,15,20,8,8,5,2,74142
74142,hsa-miR-12129,ENST00000598345,"323,336",0.538462,-17.7,0,1.63682e-05,0.353,0,0,-11.6382,12,13,12,1,4,1,74143
74143,hsa-miR-12131,ENST00000598345,"50,66",0.538462,-16.1,0,2.38062e-07,0.235,0,0,-14.9612,11,16,6,10,6,3,74144


In [40]:
#parse miRNA and mRNA seed from sequence
#miRNA_sequences.to_sql('seqs', conn, index=False, dtype={"bs_id": 'INTEGER'})

qry = '''
    alter table main 
    add column mirna_seed; 
    '''
qry2 = '''
    update main 
    set mirna_seed = ? where mirna = ?;
    '''
#c.execute(qry)
c.execute(qry2, (list(miRNA_sequences.values()), list(miRNA_sequences.keys())))
conn.commit()
data = c.fetchall()
#c.close
#conn.close()
data
#df_main['miRNA_seed'] = df_main.apply(lambda row: miRNA_sequences[row['miRNA']], axis=1)
#df_main['mRNA_bs_seq'] = df_main.apply(lambda row: mRNA_sequences[row['mRNA']][row['bs_start']:row['bs_end']], axis=1)

NameError: name 'miRNA_sequences' is not defined

In [5]:
#translate genome position relative to transcript to chromosome
def pos_to_chrom(pos, tid, ids, exon_starts, exon_ends, chrom_exon_starts):
    for j, eid in enumerate(ids[tid]):
        if tid not in exon_starts: print(tid,'not in exon_starts')
        if tid in exon_starts and eid in exon_starts[tid] and tid in exon_ends and eid in exon_ends[tid]:
            if pos >= exon_starts[tid][eid] and pos <= exon_ends[tid][eid]:
                diff = pos - exon_starts[tid][eid]
                chrom_pos = chrom_exon_starts[tid][eid] + diff
                return chrom_pos
        else:
            print(eid,'not in exon_starts')
            return None
df_main['chrom_bs_start'] = df_main.apply(lambda row: pos_to_chrom(row.bs_start, row.mRNA, ids, exon_starts, exon_ends, chrom_exon_starts), axis=1)
df_main['chrom_bs_end'] = df_main.apply(lambda row: pos_to_chrom(row.bs_end, row.mRNA, ids, exon_starts, exon_ends, chrom_exon_starts), axis=1)
df_main['bs_id'] = df_main.index
df_main.head()

,miRNA,mRNA,binding_probability,energy,seed,accessibility,AU_content,PhyloP_Stem,PyloP_Flanking,m/e,...,position_of_longest_consecutive_pairings,pairings_in_3prime_end,difference_of_pairings_between_seed_and_3prime_end,bs_start,bs_end,miRNA_seed,mRNA_bs_seq,chrom_bs_start,chrom_bs_end,bs_id
0,hsa-let-7a-5p,ENST00000600966,0.692308,-15.4,0,0.003371,0.368,-0.568203,0.198163,-7.757047,...,10,6,1,396,416,TGAGGTAGTAGGTTGTATAGTT,AACTACAGCTGCAGCTACCG,58352138,58352158,0
1,hsa-let-7a-2-3p,ENST00000600966,0.923077,-23.2,1,0.000094,0.426,0.844482,-0.095065,-5.585164,...,1,6,1,52,75,CTGTACAGCCTCCTAGCTTTCC,GGGTGGCCCAGGAGCCTGTGCAC,58350645,58351407,1
2,hsa-let-7b-5p,ENST00000600966,0.692308,-21.3,0,0.000009,0.353,1.253080,0.015255,-8.613887,...,1,7,2,775,794,TGAGGTAGTAGGTTGTGTGGTT,ACTACACCTGCCGCTACCG,58352987,58353006,2
3,hsa-let-7b-5p,ENST00000600966,0.692308,-18.5,0,0.000001,0.294,1.302461,-0.092658,-9.582385,...,1,6,1,126,143,TGAGGTAGTAGGTTGTGTGGTT,GGCCGCTACCGCTGCCG,58351458,58351475,3
4,hsa-let-7b-5p,ENST00000600966,0.846154,-19.2,0,0.000174,0.397,0.062334,0.643183,-8.600034,...,12,8,3,560,580,TGAGGTAGTAGGTTGTGTGGTT,GGCTGCACCACCACCGCCTG,58352400,58352420,4


In [11]:
#df_gene: strand - chrom - gene_id - gene_name - transcript_id (1-N)
df_gene = pd.read_csv(path2+'gene_infos.csv', dtype={'ensembl_gene_id':str,'ensembl_gene_id_version':str,'ensembl_transcript_id':str, 'ensembl_transcript_id_version':str, 'chromosome_name':'category', 'strand':'int8', 'start_position':int, 'end_position':int})
del df_gene['Unnamed: 0']
df_gene.head()

,ensembl_gene_id,ensembl_gene_id_version,ensembl_transcript_id,ensembl_transcript_id_version,chromosome_name,strand,start_position,end_position
0,ENSG00000210049,ENSG00000210049.1,ENST00000387314,ENST00000387314.1,MT,1,577,647
1,ENSG00000211459,ENSG00000211459.2,ENST00000389680,ENST00000389680.2,MT,1,648,1601
2,ENSG00000210077,ENSG00000210077.1,ENST00000387342,ENST00000387342.1,MT,1,1602,1670
3,ENSG00000210082,ENSG00000210082.2,ENST00000387347,ENST00000387347.2,MT,1,1671,3229
4,ENSG00000209082,ENSG00000209082.1,ENST00000386347,ENST00000386347.1,MT,1,3230,3304


## get TCGA data from Xena

In [30]:
import xenaPython as xena
cohort = 'TCGA PanCanAtlas'
host = xena.PUBLIC_HUBS['pancanAtlasHub']
samples = xena.cohort_samples(host, cohort, None)
samples[0: 10]
dataset = 'EB++AdjustPANCAN_IlluminaHiSeq_RNASeqV2.geneExp.xena'
expression = get_fields(host, dataset, samples, GENES) # list of lists.
expression_by_gene = dict(zip(GENES, expression))      # index by gene.
[expression_by_gene.keys(), GENES[0], expression_by_gene[GENES[0]][0:10]]
#instead of loading everything, only load for every row in bs?

C:\Users\Lena\Miniconda3\envs\thesis\lib\site-packages\xenaPython\__init__.py:110: FutureWarning: Possible nested set at position 7
  re.sub(r"^[^[]+[[]([^]]*)[]].*$", r"\1", query, flags=re.DOTALL))


NameError: name 'get_fields' is not defined

In [34]:
hub = "https://toil.xenahubs.net"
dataset = "tcga_RSEM_gene_tpm"

xena.dataset_samples(hub, dataset, None)
xena.dataset_gene_probe_avg(hub, dataset, ['TCGA-61-1910-01'], ['DUSP7'])[0]["scores"][0]

[3.469]

In [33]:
hub = "https://tcga.xenahubs.net"
dataset = "TCGA.KICH.sampleMap/miRNA_HiSeq_gene"
#samples = xena.dataset_samples (hub, dataset, None)
samples = xena.dataset_samples (hub, dataset, 10)
print(samples)
#samples = ["TCGA-02-0047-01","TCGA-02-0055-01","TCGA-02-2483-01"]
probes = ['ENSG00000282740.1', 'ENSG00000000005.5']
[position, [ENSG00000282740_1, ENSG00000000005_5]] = xena.dataset_probe_values (hub, dataset, samples, probes)
ENSG00000282740_1

['TCGA-KN-8426-01', 'TCGA-KN-8419-11', 'TCGA-KL-8332-01', 'TCGA-KN-8423-01', 'TCGA-KO-8405-01', 'TCGA-KL-8337-01', 'TCGA-KN-8424-11', 'TCGA-KN-8422-11', 'TCGA-KL-8334-01', 'TCGA-KO-8404-01']


['NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN', 'NaN']

In [9]:
%load_ext Cython

In [ ]:
%%cython
def c_split_chrom_column(chrom):
    chrom_list = chrom.split(':')
    chromosome_name = str(chrom_list[0][3:])
    strand = 1 if chrom_list[2] == '+' else -1
    chrom_exon_start = int(chrom_list[1].split('-')[0])
    chrom_exon_end = int(chrom_list[1].split('-')[1])
    return chromosome_name,strand,chrom_exon_start,chrom_exon_end

In [ ]:
def list_to_df(list_, nr_columns, data_type=None):
    data = []
    remove_last_columns = 2 if data_type != None else 0
    nr_rows = int((len(list_)-1)/nr_columns)
    for row in range(nr_rows):
        start = row*nr_columns
        end = (row+1)*nr_columns
        data.append(list_[start:end-remove_last_columns])
    df = pd.DataFrame(data[1:], columns = data[0]) #use first line as header
    if data_type=='miRNA':
        df = df.astype({'miRNA_ID':str, 'read_count':int})#, 'reads_per_million_miRNA_mapped':float, 'cross-mapped': 'category'})
    elif data_type=='exon':
        df = df.astype({'exon':str, 'raw_counts':int})#, 'reads_per_million_miRNA_mapped':float, 'cross-mapped': 'category'})
        df[['chromosome_name','strand','chrom_exon_start','chrom_exon_end']] = df.exon.apply(lambda x: pd.Series(c_split_chrom_column(str(x))))
        df[['raw_counts','chrom_exon_start','chrom_exon_end']] = df[['raw_counts','chrom_exon_start','chrom_exon_end']].apply(pd.to_numeric,downcast='unsigned')
        df[['chromosome_name','strand']] = df[['chromosome_name','strand']].astype('str').astype('category')
        del df['exon']
    return df

In [ ]:
#UCSC Xena API, get list of files which contain miRNA expression per case_id there

primary_site='Kidney'
download_files_from_legacy('miRNA',3000,primary_site,write=True) #2029 #70min
#download_files_from_legacy('exon',2000,primary_site,write=True) #1588 #35h

In [6]:
#read in downloaded counts from feather files
#TODO remove later when all data there
n_all_mirna_dfs = {}
dir_ = os.path.join(path_gdc,'mirna')
for i, file in enumerate(os.listdir(dir_)):
    #print(file)
    filename = ('.').join(file.split('.')[0:3])
    n_all_mirna_dfs[filename] = pd.read_feather(os.path.join(dir_,file))
    
n_all_exon_dfs = {}
dir_ = os.path.join(path_gdc,'exon')
for i, file in enumerate(os.listdir(dir_)):
    #print(file)
    filename = ('.').join(file.split('.')[0:3])
    n_all_exon_dfs[filename] = pd.read_feather(os.path.join(dir_,file))
print(len(n_all_exon_dfs),len(n_all_mirna_dfs)) #yes this is right, read in from path

1588 2013


## Map exons to bs

In [19]:
new_exon_counts = n_all_exon_dfs
new_exon_counts.head()

,chromosome_name,chrom_exon_start,chrom_exon_end,strand,raw_counts
index,,,,,
0,1,11874,12227,1,15.0
1,1,12595,12721,1,3.0
2,1,12613,12721,1,3.0
3,1,12646,12697,1,2.0
4,1,13221,14409,1,67.0


In [12]:
important_counts = df_main[['bs_id','miRNA','mRNA','chrom_bs_start','chrom_bs_end']].copy()
important_counts = important_counts.merge(df_gene[['ensembl_transcript_id','chromosome_name','strand']], left_on='mRNA', right_on='ensembl_transcript_id', how='left')
important_counts[['chromosome_name','strand','miRNA','mRNA']] = important_counts[['chromosome_name','strand','miRNA','mRNA']].astype('str').astype('category')
del important_counts['ensembl_transcript_id']

In [28]:
#for a large data set, you will likely get a significant speed increase by creating an index for any column(s) used in the join condition. 
#https://stackoverflow.com/questions/30627968/merge-pandas-dataframes-where-one-value-is-between-two-others
#instead use SQL
conn = sqlite3.connect(':memory:') #Make the database in memory

#write the tables
important_counts.to_sql('bs', conn, index=False, dtype={"bs_id": 'INTEGER'})
new_exon_counts.to_sql('counts', conn, index=True, index_label='exon_id', dtype={"raw_counts": 'INTEGER', "exon_id": 'INTEGER'})
qry = '''
    select
        counts.chromosome_name, counts.chrom_exon_start, counts.chrom_exon_end, counts.strand, counts.raw_counts, counts.exon_id, bs.bs_id, bs.miRNA, bs.mRNA, bs.chrom_bs_start, bs.chrom_bs_end
    from
        bs inner join counts on
        bs.chromosome_name = counts.chromosome_name and bs.strand = counts.strand and bs.chrom_bs_start >= counts.chrom_exon_start and bs.chrom_bs_end <= counts.chrom_exon_end
    '''
bs_exons = pd.read_sql_query(qry, conn) #only return bs with minimum 1 exon_id cause inner join (else left join)
bs_exons.head()
#TODO filter out all binding sites that have more than 1 exon_id: none for now

,chromosome_name,chrom_exon_start,chrom_exon_end,strand,raw_counts,exon_id,bs_id,miRNA,mRNA,chrom_bs_start,chrom_bs_end
0,19,58352928,58353197,-1,0,216603,2,hsa-let-7b-5p,ENST00000600966,58352987,58353006
1,19,58351391,58351687,-1,0,216600,3,hsa-let-7b-5p,ENST00000600966,58351458,58351475
2,19,58352283,58352555,-1,0,216602,4,hsa-let-7b-5p,ENST00000600966,58352400,58352420
3,19,58351391,58351687,-1,0,216600,5,hsa-let-7b-3p,ENST00000600966,58351648,58351684
4,19,58351391,58351687,-1,0,216600,8,hsa-let-7d-5p,ENST00000600966,58351399,58351420


In [158]:
#visualize amount exons by bindingsite | amount bindingsites by exon
import sqlite3
conn = sqlite3.connect(':memory:') #Make the database in memory

#write the tables
important_counts.to_sql('bs', conn, index=False, dtype={"bs_id": 'INTEGER'})
new_exon_counts.to_sql('counts', conn, index=True, index_label='exon_id', dtype={"exon_id": 'INTEGER'})
qry = '''
    select
        counts.chromosome_name, counts.chrom_exon_start, counts.chrom_exon_end, counts.strand, counts.raw_counts, counts.exon_id, bs.bs_id, bs.miRNA, bs.mRNA, bs.chrom_bs_start, bs.chrom_bs_end
    from
        bs left join counts on
        bs.chromosome_name = counts.chromosome_name and bs.strand = counts.strand and bs.chrom_bs_start >= counts.chrom_exon_start and bs.chrom_bs_end <= counts.chrom_exon_end
    '''
bs_exons2 = pd.read_sql_query(qry, conn)
bs_exons2.head()
#bs_exons2.groupby('bs_id').count().raw_counts.hist()
#bs_exons2.groupby('exon_id').count()

## Map miRNA to bs

In [29]:
#map miRNA ID to mature ID
mirna_counts = all_mirna_dfs[case_id]

trans = pd.DataFrame.from_dict(read_in_miRNA(path2+'miRNA2mature.fasta'), orient='index')#,columns=["mirna", "mature"])
trans = trans.rename(columns={0: "miRNA_ID"})
trans[['miRNA_ID']] = trans[['miRNA_ID']].astype('str').astype('category')

#map miRNA to miRNA family
mirna_families = read_in_mirna_families(path_tarp+'mapping_miRNA_families.fasta')
trans = trans.join(mirna_families,how='inner')

bs_mirnas = bs_exons[['exon_id','raw_counts','miRNA']].join(trans,on='miRNA').merge(mirna_counts[['miRNA_ID','read_count']],on='miRNA_ID')
#bs_mirnas.drop(['miRNA','miRNA_ID'], axis=1, inplace=True)
bs_mirnas = bs_mirnas.rename(columns={'read_count': "mirna_read_count",'raw_counts':'exon_raw_counts','miRNA family name':'mirna_family'})
bs_mirnas[['mirna_read_count']] = bs_mirnas[['mirna_read_count']].astype('int')#TODO do this early before saving
bs_mirnas.head()

#2693 with hsa in miRNA2mature / 49156 overall in miRNA2mature (also other species eg cel)
#2656 if inner join cause only 2656 families
#TODO check if miRNA counts from TCGA are more different miRNAs?
#TODO is inner right? are some in important counts None?

KeyError: '24ea3cd5-c2e6-44f7-9658-78af7befeef7'

In [227]:
bs_mirnas[bs_mirnas.exon_id ==216595]

,exon_id,exon_raw_counts,miRNA,miRNA_ID,mirna_family,mirna_read_count
7,216595,0,hsa-let-7b-5p,hsa-let-7b,let-7,55647
14,216595,0,hsa-let-7d-5p,hsa-let-7d,let-7,3767
25,216595,0,hsa-let-7e-3p,hsa-let-7e,let-7,11424
33,216595,0,hsa-let-7e-3p,hsa-let-7e,let-7,11424
46,216595,0,hsa-miR-15a-3p,hsa-mir-15a,mir-15,2258
...,...,...,...,...,...,...
12562,216595,0,hsa-miR-5692a,hsa-mir-5692a-2,mir-5692,0
12563,216595,0,hsa-miR-548az-5p,hsa-mir-548az,mir-548,0
12566,216595,0,hsa-miR-8062,hsa-mir-8062,hsa-miR-8062,0
12571,216595,0,hsa-miR-8064,hsa-mir-8064,hsa-miR-8064,0


In [226]:
bs_mirnas[bs_mirnas.mirna_family =='hsa-miR-1229-5p']

# 1 right? count is added for exonid 216595, but do i already add it before ?
# 2 also : exon_id + mirna double (3368 & 3370) wrong!!

,exon_id,exon_raw_counts,miRNA,miRNA_ID,mirna_family,mirna_read_count
3365,216603,0,hsa-miR-1229-5p,hsa-mir-1229,hsa-miR-1229-5p,4
3368,216595,0,hsa-miR-1229-5p,hsa-mir-1229,hsa-miR-1229-5p,4
3370,216595,0,hsa-miR-1229-5p,hsa-mir-1229,hsa-miR-1229-5p,4
3371,216596,0,hsa-miR-1229-5p,hsa-mir-1229,hsa-miR-1229-5p,4


In [ ]:
#output: for each case_id for each bs_id: interesting miRNA expression + exon expression
#unstack family counts from 2 columns to several columns (mirna_family=name,mirna_read_count=value)
pivoted = bs_mirnas.copy()
pivoted = pivoted.pivot_table(values='mirna_read_count', index=['exon_id', 'exon_raw_counts'], columns='mirna_family', aggfunc='sum', fill_value=0)
pivoted#.head()

In [ ]:
#visualize amount exons by bindingsite
#bs_exons.groupby('bs_id').count().raw_counts.hist()

In [274]:
#TODO check - is nr mirna downloaded also here? if not: mistake
print(X.shape,y[0].shape)

(8, 1680) (8,)


## Elastic Net Regression

In [ ]:
import sys
sys.getsizeof(pivoted)#TODO

In [276]:
#regression btw miRNA expression, exon expression
import sklearn
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
#faster if np.array(x, order='F')
#If you are interested in controlling the L1 and L2 penalty separately, keep in mind that this is equivalent to:
#a * L1 + b * L2 where: alpha = a + b and l1_ratio = a / (a + b)
X = np.array(pivoted)
y = np.array(pivoted.index.get_level_values('exon_raw_counts'))

regr = ElasticNet(random_state=0)
regr.fit(X, y)
print(regr.coef_)
print(regr.intercept_)

[0. 0. 0. ... 0. 0. 0.]
0.0


C:\Users\Lena\Miniconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0, tolerance: 0.0
  model = cd_fast.enet_coordinate_descent(


## Get exonid for exon for database later

In [ ]:
#TODO this is not necessary anymore if the new exon counts work!!!!
#instead try directly with ensembl start & end match - if it doesnt work 
#-> try download from https://www.gencodegenes.org/human/ (GRCh38.p13) if same reference genome

In [23]:
#read in exon ids + starts and ends from GENCODE GTF file (ca 12min)
#import the file into a local sqlite3 file-based database
import gffutils
fn = gffutils.example_filename(path2+'gencode.v19.annotation.gtf_withproteinids')
db = gffutils.create_db(fn, ':memory:', keep_order=True, disable_infer_genes=True, disable_infer_transcripts=True)

In [20]:
new_exon_counts['exonname'] = np.nan 

In [27]:
for exon in db.features_of_type('exon'):
    print(exon)#TODO
    is_same = (new_exon_counts.chromosome_name == exon.chromosome) and (new_exon_counts.chrom_exon_start == exon.start) and (new_exon_counts.chrom_exon_end == exon.stop) and (new_exon_counts.strand == exon.strand)
    new_exon_counts[is_same].exonname = exon.exon_id
    print(len(new_exon_counts[is_same]))
    #exons = list(db.children(gene, featuretype='exon'))
    #first_3_exons = exons[:3]
    #last_3_exons = exons[-3:]

chr1	HAVANA	exon	11869	12227	.	+	.	gene_id "ENSG00000223972.4"; transcript_id "ENST00000456328.2"; gene_type "pseudogene"; gene_status "KNOWN"; gene_name "DDX11L1"; transcript_type "processed_transcript"; transcript_status "KNOWN"; transcript_name "DDX11L1-002"; level "2"; havana_gene "OTTHUMG00000000961.2"; tag "basic"; havana_transcript "OTTHUMT00000362751.1"; exon_number "1"; exon_id "ENSE00002234944.1";


AttributeError: 'Feature' object has no attribute 'chromosome'

In [ ]:
#exon line
#chr1	HAVANA	exon	11869	12227	.	+	.	gene_id "ENSG00000223972.4"; 
#transcript_id "ENST00000456328.2"; gene_type "pseudogene"; gene_status "KNOWN"; 
#gene_name "DDX11L1"; transcript_type "processed_transcript"; transcript_status "KNOWN"; 
#transcript_name "DDX11L1-002"; exon_number 1; exon_id "ENSE00002234944.1"; level 2; tag "basic"; 
#havana_gene "OTTHUMG00000000961.2"; havana_transcript "OTTHUMT00000362751.1";